In [19]:
pip install d2l

# Classification 

## 载入数据集

In [20]:
import torch
import torchvision
from torch.utils import data
from torchvision import transforms 
from d2l import torch as d2l
d2l.use_svg_display()

In [21]:
trans = transforms.ToTensor()
mnist_train = torchvision.datasets.FashionMNIST(
root="../data", train=True, transform=trans, download=True) 
mnist_test = torchvision.datasets.FashionMNIST(
root="../data", train=False, transform=trans, download=True)

In [22]:
len(mnist_train), len(mnist_test)

(60000, 10000)

In [23]:
mnist_train[0][0].shape

torch.Size([1, 28, 28])

In [24]:
def get_fashion_mnist_labels(labels): 
  """返回Fashion-MNIST数据集的文本标签"""
  text_labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat',
'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot'] 
  return [text_labels[int(i)] for i in labels]

In [25]:
batch_size = 256
def get_dataloader_workers():   
  return 4
train_iter = data.DataLoader(mnist_train, batch_size, shuffle=True, num_workers=get_dataloader_workers())

In [26]:
timer = d2l.Timer() 
for X, y in train_iter:
  continue
f'{timer.stop():.2f} sec'

'9.33 sec'

In [27]:
def load_data_fashion_mnist(batch_size, resize=None): 
  trans = [transforms.ToTensor()]
  if resize:
    trans.insert(0, transforms.Resize(resize))
  trans = transforms.Compose(trans)
  mnist_train = torchvision.datasets.FashionMNIST(
      root="../data", train=True, transform=trans, download=True) 
  mnist_test = torchvision.datasets.FashionMNIST(
      root="../data", train=False, transform=trans, download=True) 
  return (data.DataLoader(mnist_train, batch_size, shuffle=True, num_workers=get_dataloader_workers()), data.DataLoader(mnist_test, batch_size, shuffle=False, num_workers=get_dataloader_workers()))

In [28]:
train_iter, test_iter = load_data_fashion_mnist(32, resize=64) 
for X, y in train_iter:
  print(X.shape, X.dtype, y.shape, y.dtype) 
  break

torch.Size([32, 1, 64, 64]) torch.float32 torch.Size([32]) torch.int64


## Softmax Torch

In [29]:
import torch
from torch import nn
from d2l import torch as d2l

- 1.载入数据集

In [30]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

- 2.定义神经网络

nn.Flatten 默认按照0维铺平

In [31]:
net = nn.Sequential(nn.Flatten(), nn.Linear(784, 10))

- 3.定义weight和bias

In [32]:
def init_weights(m):
  if type(m) == nn.Linear:
    nn.init.normal_(m.weight, std=0.01)
net.apply(init_weights)

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
)

- 4.定义loss

In [33]:
loss = nn.CrossEntropyLoss()

- 5.定义优化器

In [34]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

- 6.Training and Prediction

In [35]:
def acc(X,y,net):
  p_predict = net(X)
  y_predict = torch.argmax(p_predict,axis=1)
  output = y_predict - y
  right_pre = len(output[output==0])
  right_accu = right_pre/len(output)
  return right_accu

In [36]:
epochs = 20
for epoch in range(epochs):

  for X,y in train_iter:

    l = loss(net(X), y)
    optimizer.zero_grad()
    l.backward()
    optimizer.step()

  l = loss(net(X), y)

  train_batch = 0
  train_accu = 0
  test_batch = 0
  test_accu = 0

  for X_train,y_train in train_iter:
    train_accu += acc(X_train, y_train, net)
    train_batch += 1
    train_avg_acc = train_accu/train_batch

  for X_test,y_test in test_iter:
    test_accu += acc(X_test, y_test, net)
    test_batch += 1
    avg_acc = test_accu/test_batch

  print(f'epoch {epoch + 1}, loss {l:f}, train_accu {train_avg_acc:f}, test_accu {avg_acc:f}'

epoch 1, loss 0.978840, train_accu 0.689439, test_accu 0.678418
epoch 2, loss 0.773541, train_accu 0.736137, test_accu 0.724414
epoch 3, loss 0.783152, train_accu 0.757657, test_accu 0.747266
epoch 4, loss 0.674487, train_accu 0.771714, test_accu 0.760938
epoch 5, loss 0.552532, train_accu 0.782558, test_accu 0.770215
epoch 6, loss 0.726093, train_accu 0.789290, test_accu 0.780176
epoch 7, loss 0.583779, train_accu 0.796858, test_accu 0.786426
epoch 8, loss 0.680089, train_accu 0.800460, test_accu 0.790723
epoch 9, loss 0.631717, train_accu 0.804211, test_accu 0.792480
epoch 10, loss 0.533000, train_accu 0.806632, test_accu 0.796973
epoch 11, loss 0.604920, train_accu 0.810577, test_accu 0.798145
epoch 12, loss 0.585606, train_accu 0.813154, test_accu 0.801953
epoch 13, loss 0.713481, train_accu 0.815237, test_accu 0.804297
epoch 14, loss 0.612652, train_accu 0.816955, test_accu 0.805957
epoch 15, loss 0.531944, train_accu 0.818357, test_accu 0.807520
epoch 16, loss 0.709812, train_acc